In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("data/insurance.txt")
df.head()


: 

In [ ]:
df.info()


In [ ]:
cat_cols = [
    "IsVATRegistered","Citizenship","LegalType","Title","Language",
    "Bank","AccountType","MaritalStatus","Gender","Country","Province",
    "PostalCode","MainCrestaZone","SubCrestaZone","ItemType","VehicleType",
    "Make","Model","Bodytype","NumberOfDoors","NewVehicle","CrossBorder",
    "CoverCategory","CoverType","CoverGroup","Section","Product",
    "StatutoryClass","StatutoryRiskType"
]

for col in cat_cols:
    df[col] = df[col].astype('category')
df["TransactionMonth"] = pd.to_datetime(df["TransactionMonth"])


In [ ]:
df.isnull().sum().sort_values(ascending=False)
sns.boxplot(x=df["TotalClaims"])
plt.xscale("log")


In [ ]:
df["LossRatio"] = df["TotalClaims"] / df["TotalPremium"]
overall_loss_ratio = df["LossRatio"].mean()
overall_loss_ratio
loss_by_prov = df.groupby("Province").agg(
    Claims=("TotalClaims","sum"),
    Premium=("TotalPremium","sum")
)
loss_by_prov["LossRatio"] = loss_by_prov["Claims"] / loss_by_prov["Premium"]
loss_by_prov.sort_values("LossRatio", ascending=False)
loss_by_gender = df.groupby("Gender").agg(
    Claims=("TotalClaims","sum"),
    Premium=("TotalPremium","sum")
)
loss_by_gender["LossRatio"] = loss_by_gender["Claims"] / loss_by_gender["Premium"]
loss_by_gender
loss_by_vehicle = df.groupby("VehicleType").agg(
    Claims=("TotalClaims","sum"),
    Premium=("TotalPremium","sum")
)
loss_by_vehicle["LossRatio"] = loss_by_vehicle["Claims"] / loss_by_vehicle["Premium"]
loss_by_vehicle.sort_values("LossRatio", ascending=False)


In [ ]:
sns.histplot(df["TotalPremium"], bins=50)
plt.title("Distribution of Total Premium")
plt.show()
sns.histplot(df["TotalClaims"], bins=50)
plt.yscale("log")
plt.title("Distribution of Total Claims (Log Scale)")
plt.show()
sns.histplot(df["CustomValueEstimate"], bins=50)
plt.title("Vehicle Value Distribution")
plt.show()


In [ ]:
df["Month"] = df["TransactionMonth"].dt.to_period("M")
monthly = df.groupby("Month")["TotalClaims"].sum()
monthly.plot(kind="line", figsize=(10,4), title="Monthly Total Claims")
plt.show()
df["HasClaim"] = df["TotalClaims"] > 0
freq = df.groupby("Month")["HasClaim"].mean()
freq.plot(kind="line", figsize=(10,4), title="Monthly Claim Frequency")
plt.show()


In [ ]:
vehicle = df.groupby("Make").agg(
    Claims=("TotalClaims","sum"),
    Premium=("TotalPremium","sum")
)
vehicle["LossRatio"] = vehicle["Claims"] / vehicle["Premium"]
vehicle.sort_values("LossRatio", ascending=False).head(10)
top = vehicle.sort_values("LossRatio").tail(10)
sns.barplot(x=top["LossRatio"], y=top.index)
plt.title("Top 10 High-Risk Vehicle Makes")
plt.show()


In [ ]:
numeric_cols = ["TotalPremium","TotalClaims","CustomValueEstimate","Kilowatts","Cubiccapacity"]
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap="coolwarm")
plt.show()
sns.scatterplot(x="TotalPremium", y="TotalClaims", data=df)
plt.title("Premium vs Claims")
plt.show()


In [ ]:
sns.barplot(data=loss_by_prov.reset_index(), x="LossRatio", y="Province")
plt.title("Loss Ratio by Province")
plt.show()
sns.scatterplot(data=df, x="CustomValueEstimate", y="TotalClaims")
plt.xscale("log")
plt.yscale("log")
plt.title("Vehicle Value vs Claim Amount")
plt.show()
freq.plot(kind="line", marker="o", figsize=(10,4), color="purple")
plt.title("Monthly Claim Frequency Trend")
plt.ylabel("Frequency (%)")
plt.show()


In [ ]:
Q1 = df['claim_amount'].quantile(0.25)
Q3 = df['claim_amount'].quantile(0.75)
IQR = Q3-Q1
outliers = df[(df['claim_amount'] < Q1 - 1.5*IQR) | (df['claim_amount'] > Q3 + 1.5*IQR)]
len(outliers)
